<a href="https://colab.research.google.com/github/muluwork-shegaw/Moodle-Database-Educational-Data-Log-Analysis-/blob/master/Educational_Data_Log_Analysis_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")
%cd "drive/My Drive/"

Mounted at /content/drive/
/content/drive/My Drive
